<a href="https://colab.research.google.com/github/EduardMozharov/ML_Neural_networks/blob/main/Project_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project "Emotion Detection"
Итоговая работа курса SKILLBOX:  
Data Science. ML. Средний уровень (Нейронны сети))

# Задание
**Цель** - реализовать нейронную сеть, распознающую эмоции

**Задачи:**
1. Работа требует реализации алгоритма, с помощью которого будет происходить заданная классификация. Приветствуется использование архитектур свёрточных нейронных сетей, разобранных во время теоретических занятий. Время инференса сети на Google Colab не должно превышать 0,33 секунды (3 кадра в секунду).

2. Создать скрипт, который будет работать с их веб-камерами и выводить на экран текущую эмоцию.


# 1 Загрузим необходимые библиотеки

In [ ]:
!pip install livelossplot

# Для работы оптимизатора TensorRT может потребоваться предыдущая версия tensorflow
#!pip install -U tensorflow==2.7.0
#!pip install keras==2.6.*

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print('TensorFlow version:', tf.__version__)
from livelossplot.tf_keras  import PlotLossesCallback

import timeit

TensorFlow version: 2.9.2


#2. Загрузим данные и рассмотрим их

#3. Модель

#4. Обучение модели